<font size="20">

# LET measurements

<font size="3">
    
> __Author:__ Alexander Backis
<br/>__Institutes:__ University of Glasgow, European Spallation Source
<br/>__Dates:__ 2/10-2020 to 16/10-2020

_Abstract:_
This notebook contains the data-analysis tools used for the measurements at the LET instrument at ISIS. It describes how the analysis was performed, and summarizes the results.

# Contents
    
* [1. Introduction](#INTRODUCTION)
    * [1.1 Packages](#PACKAGES)
    * [1.2 Parameters](#PARAMETERS)
    * [1.3 Functions](#FUNCTIONS)
* [2. Data](#DATA)
    * [2.1 Paths](#PATHS)
    * [2.2 Extract](#EXTRACT)
    * [2.3 Load](#LOAD)
* [3. Filters](#FILTER)
* [4. Plotting](#PLOTTING)
* [5. Distance calibration](#CALIBRATION)

# 1. Introduction

Notebook showing the analysis of the LET measurements.

## 1.1 Packages<a class="anchor" id="PACKAGES"></a>

Import all necessary packages.

In [ ]:
# Autoload packages when doing an external change
%load_ext autoreload
%autoreload 2

# Activate matplotlib in interactive notebook mode
%matplotlib notebook

# General packages
import os
import h5py
import numpy as np
import pandas as pd
import plotly as py
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import plotly.graph_objects as go
import scipy.optimize as sci_opt
import scipy.signal as sci_sig
from plotly.offline import iplot, plot

# Data handling
import mg.file_handling.read as mg_read
import he3_let.read as he3_read

# Plotting
import mg_let.basic_plot as mg_basic_plot

# Helper functions
import mg.helper_functions.misc as mg_hf

## 1.2 Parameters

In [ ]:
# Declare all global parameters
MG_RAW_FOLDER = '../data/let_measurements/mg/raw/'
MG_PROCESSED_FOLDER = '../data/let_measurements/mg/processed/'
HE3_FOLDER = '../data/let_measurements/he3/'

# Instrument definition
MODERATOR_TO_SAMPLE_IN_M = 25 # m

# Convertion factors
MG_TDC_TO_US = 62.5e-9 * 1e6

## 1.3 Functions

Declare all functions used in the analysis.

## Multi-Grid detector

### Plotting

In [ ]:
def mg_plot_basic(run, clusters_unfiltered, events, df_filter, bus_start, bus_stop, number_rows, area):
    """
    Function to plot all basic plots for SEQUOIA detector,
    such as PHS, Coincidences and rate.
    
    Ordering of plotting is:
    
    PHS 2D - NOT FILTERED
    MULTIPLICITY - FILTERED
    PHS 1D - FILTERED
    COINCIDENCES 2D - FILTERED
    PHS CORRELATION - FILTERED
    RATE - FILTERED
    TOF - FILTERED
    
    Note that all plots are filtered except the PHS 2D.
    
    Args:
        run (str): File run
        clusters_unfiltered (DataFrame): Unfiltered clusteres
        events (DataFrame): Individual events
        df_filter (dict): Dictionary specifying the filter which will be used on the clustered data
        bus_start (int): First bus to plot
        bus_stop (int): Last bus to plot
        number_rows (int): Number of rows in plots (number of rows must be larger than number_buses/3)
        area (float): Area in m^2 of the active detector surface

    Yields:
        Plots the basic analysis
    
    """
    
    mg_hf.set_thick_labels(12)
    
    # Filter clusters
    clusters = mg_read.filter_data(clusters_unfiltered, df_filter)
    
    # Declare parameters
    duration = (clusters_unfiltered.time.values[-1] - clusters_unfiltered.time.values[0]) * 62.5e-9
    
    # PHS - 2D
    vmin = 1
    vmax = events.shape[0] // 1000 + 100
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        events_bus = events[events.bus == bus]
        if events_bus.shape[0] > 0:
            mg_basic_plot.phs_2d_plot(events_bus, bus, vmin, vmax)
    plt.tight_layout()
    
    # Save data
    output_path = '../output/%s_phs_2d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Multiplicity
    vmin = None
    vmax = None
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        if clusters_bus.shape[0] > 1:
            mg_basic_plot.multiplicity_plot_perc(clusters_bus, bus, duration)
    plt.tight_layout()
    
    # Save data
    output_path = '../output/%s_multiplicity_2d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # PHS - 1D
    vmin = None
    vmax = None
    bins_phs_1d = 300
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        clusters_uf_bus = clusters_unfiltered[clusters_unfiltered.bus == bus]
        mg_basic_plot.phs_clusters_1d_plot(clusters_bus, clusters_uf_bus, bins_phs_1d, bus, duration)
        plt.yscale('log')
    plt.tight_layout()
    
    # Save data
    output_path = '../output/%s_phs_1d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Coincidences - 2D
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    if clusters.shape[0] != 0:
        vmin = (1 * 1/duration)
        vmax = (clusters.shape[0] // 450 + 5) * 1/duration
    else:
        duration = 1
        vmin = 1
        vmax = 1
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        # Calculate number of events and rate in a specific bus
        number_events = clusters_bus.shape[0]
        events_per_s = number_events/duration
        events_per_s_m2 = events_per_s/area
        title = ('Bus %d\n(%d events, %.6f events/s/m$^2$)' % (bus, number_events, events_per_s_m2))
        if number_events > 1:
            mg_basic_plot.clusters_2d_plot(clusters_bus, title, vmin, vmax, duration)
    plt.tight_layout()
    
    # Save data
    output_path = '../output/%s_coincidences_2d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Coincidences - PHS
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    if clusters.shape[0] != 0:
        vmin = 1/duration
        vmax = (clusters.shape[0] // 450 + 1000) / duration
    else:
        duration = 1
        vmin = 1
        vmax = 1
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        if clusters_bus.shape[0] > 1:
            mg_basic_plot.clusters_phs_plot(clusters_bus, bus, duration, vmin, vmax)
    plt.tight_layout()
    
    # Save data
    output_path = '../output/%s_coincidences_phs.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Rate 
    number_bins = 50
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        mg_basic_plot.rate_plot(clusters_bus, number_bins, bus, area)
    plt.tight_layout()
    
    # Save data
    output_path = '../output/%s_rate.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # TIME-OF-FLIGHT
    number_bins = 300
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        mg_basic_plot.tof_histogram(clusters_bus, number_bins, bus)
    plt.tight_layout()
    
    # Save data
    output_path = '../output/%s_tof.png' % run
    fig.savefig(output_path, bbox_inches='tight')

### Normalization

### Distance calibration

In [ ]:
def distance_calibration(df, E_in_meV, number_bins=100):
    """
    Function to calibrate the distance to each voxel in the MG from the sample. Data is collected
    from a monochromatic neutron beam incident on a small vanadium sample. The script histograms
    data from each voxel, takes the tof corrsponding to the peak maximum, and calculates the 
    sample-detector distance based on this.
    
    Args:
        df (pd.DataFrame): DataFrame containing the Multi-Grid data
        E_in_meV (float): Incident energy of neutrons
        number_bins (int): Number of bins for histogram

    Returns:
        distances (np.array): A 3D-array containing the '(bus, wch, gch) -> distance'-mapping
    
    """
    # Define parameters
    buses = np.array([0, 1])
    whcs = np.arange(0, 80, 1)
    gchs = np.arange(80, 120, 1)
    # Calculate velocity of incident neutrons
    v_in_m_per_s = np.sqrt((E_in_meV/5.227)) *1e3 # From Squires (p. 3)
    tof_moderator_to_sample_in_s = (MODERATOR_TO_SAMPLE_IN_M/v_in_m_per_s)
    # Declare 3-dimensional numpy array
    distances = np.zeros((len(buses), 80, 120), dtype='float')
    # Iterate through all voxels
    for bus in buses:
        for wch in whcs:
            for gch in gchs:
                # Get data from a single voxel
                voxel_data = df[(df.bus == bus) &
                                (df.wch == wch) &
                                (df.gch == gch)]
                # Histogram tof data
                hist, bin_edges = np.histogram(voxel_data['tof'], bins=number_bins)
                bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
                # Get tof corresponding to peak maximum
                idx_max_value = np.argmax(hist)
                tof_in_s = bin_centers[idx_max_value] * 1e-6
                # Get distance corresponding to tof
                tof_sample_to_detection_in_s = tof_in_s - tof_moderator_to_sample_in_s
                distance_in_m = v_in_m_per_s * tof_sample_to_detection_in_s
                # Save in distances 3D-matrix
                distances[bus, wch, gch] = distance_in_m
    return distances


## Helium-3 tubes

### Plotting

In [ ]:
def he3_plot_coordinates(pixels, region=None):
    """
    Function to plot the positions of all the pixels in the LET helium-3 array, as well as
    highlighting a region of interest.
    
    Args:
        pixels (np.array): Pixel coordinates in Helium-3 array
        region (np.array): Pixel ID's corresponding to a small region in the Helium-3 array
                           which is to be highlighted.

    Yields:
        A 3D plot showing all the pixels in the helium-3 array (blue), as well highlightning
        the position of the pixel id's provided in 'region' (green), and showing the sample
        position (red).
    
    """
    # Define labels
    labels = []
    for i in np.arange(0, len(pixels['x']), 1):
        labels.append('ID: %d<br>θ: %.2f°<br>φ: %.2f°' % (i, pixels['theta'][i], pixels['phi'][i]))
    trace_1 = go.Scatter3d(
        x=pixels['x'],
        y=pixels['y'],
        z=pixels['z'],
        mode='markers',
        marker=dict(
            size=4,
            line=dict(
                color='rgba(0, 0, 255, 1)',
                width=1.0
            ),
            opacity=1.0
        ),
        name='Helium-3 tubes',
        text=labels
    )
    
    trace_2 = go.Scatter3d(
        x=[0],
        y=[0],
        z=[0],
        mode='markers',
        marker=dict(
            size=4,
            line=dict(
                color='rgba(255, 0, 0, 1)',
                width=5.0
            ),
            opacity=1.0
        ),
        name='Sample'
    )
    
    if region is not None:
        # Define labels
        labels = []
        for i in region:
            labels.append('ID: %d<br>θ: %.2f°<br>φ: %.2f°' % (i, pixels['theta'][i], pixels['phi'][i]))
        trace_3 = go.Scatter3d(
                            x=pixels['x'][region],
                            y=pixels['y'][region],
                            z=pixels['z'][region],
                            mode='markers',
                            marker=dict(
                                    size=4,
                                    line=dict(
                                            color='rgba(0, 255, 0, 1)',
                                            width=1.0
                                            ),
                                    opacity=1.0
                            ),
                            name='Multi-Grid sized region',
                            text=labels
                            )

        
    
    camera = dict(
        up=dict(x=0, y=1, z=0),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=-1.5, y=1.5, z=-1.5)
    )

    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        scene=dict(
                camera=camera,
                xaxis = dict(
                        title='x (m)'),
                yaxis = dict(
                        title='y (m)'),
                zaxis = dict(
                        title='z (m)'),)
    )
    py.offline.init_notebook_mode()
    if region is None:
        data = [trace_1, trace_2]
    else:
        data = [trace_1, trace_2, trace_3]
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [ ]:
def plot_helium_3_(pixels, region=None):
    

### Normalization

In [ ]:
def he3_get_area(number_pixels, pixels_per_tube=256):
    """
    Get area covered by the specified number of pixels in the helium-3 tubes.
    
    Args:
        number_pixels (np.array): Number of pixels
        pixels_per_tube (int): Number of pixels per tube

    Return:
        area_in_m2 (float): Area in m^2 covered by pixels
    """
    tube_length = 4 # [m]
    tube_diameter = 0.0254 # [m]
    segment_area = (tube_length*tube_diameter)/pixels_per_tube
    area_in_m2 = segment_area * number_pixels
    return area_in_m2

def he3_get_solid_angle(pixels, pixels_per_tube=256):
    """
    Get solid angle covered by the specified pixels in the helium-3 tubes.
    
    Args:
        pixels (np.array): Pixel coordinates in Helium-3 array
        pixels_per_tube (int): Number of pixels per tube

    Return:
        solid_angle (float): Solid angle in radians covered by pixels
    """
    # Get area of one pixel
    pixel_area = he3_get_area(1)
    # Get solid angle by summing contribution from each pixel
    solid_angle = 0
    for r, theta, phi in zip(pixels['r'], pixels['theta'], pixels['phi']):
        projected_area = pixel_area * np.cos(phi * ((2*np.pi)/360))
        pixel_solid_angle = projected_area / (r ** 2)
        solid_angle += pixel_solid_angle
    return solid_angle

def get_duration(path):
    """
    Get measurement duration of file.
    
    Args:
        path (str): Path to '.nxs'-file.

    Return:
        duration_in_s (float): Duration of measurements in seconds
    """
    nxs = h5py.File(path, 'r')
    duration_in_s = nxs['raw_data_1']['duration'][0] # In seconds
    return duration_in_s
    

## Common

### Energy calculation

In [ ]:
def get_energy(tof_in_us, sample_to_detection_in_m):
    """
    Function to calculate neutron energy by first calculating neutron velocity
    from time-of-flight and flight distance.
    
    Args:
        tof_in_us (np.array): Time-of-flight in micro seconds
        sample_to_detection_in_m (np.array): Sample-to-detection distance in meters

    Return:
        energies_in_meV (np.array): Neutron energies
    """
    # Energy calculation
    full_distance = MODERATOR_TO_SAMPLE_IN_M + sample_to_detection_in_m
    v_in_km_per_s = (full_distance * 1e-3)/(tof_in_us * 1e-6)
    energies_in_meV = 5.227 * (v_in_km_per_s ** 2) # Calculated according to Squires (p. 3)
    return energies_in_meV

### Energy transfer

In [ ]:
def get_energy_transfer(Ei_in_meV, tof_in_us, sample_to_detection_in_m):
    """
    Function to calculate neutron energy transfer by calculating the initial neutron velocity, v_i, 
    as well as the final neutron velocity, v_f, after the scattering.
    
    Args:
        Ei_in_meV (float): Initial energy in meV
        tof_in_us (np.array): Time-of-flight in micro seconds
        sample_to_detection_in_m (np.array): Sample-to-detection distance in meters

    Return:
        delta_E (np.array): Neutron energy transfer (Ei - Ef)
    """
    
    # Get initial neutron velocity
    vi_in_m_per_s = np.sqrt((Ei_in_meV/5.227)) * 1e3 # From Squires (p. 3)
    # Get final neutron velocity
    tof_moderator_to_sample_in_s = (MODERATOR_TO_SAMPLE_IN_M/vi_in_m_per_s)
    tof_sample_to_detection_in_s = (tof_in_us * 1e-6) - tof_moderator_to_sample_in_s
    vf_in_km_per_s = (sample_to_detection_in_m/tof_sample_to_detection_in_s) * 1e-3
    # Get final neutron energy
    Ef_in_meV = 5.227 * (vf_in_km_per_s ** 2) # Calculated according to Squires (p. 3)
    # Calculate energy transfer
    delta_E = Ei_in_meV - Ef_in_meV
    return delta_E

### Gaussian fitting

In [ ]:
# Perform Gaussian fitting procedure
def fit_data(hist, bins, a_guess, x0_guess, sigma_guess):
    """
    Function to fit a Gaussian distribution.
    
    Args:
        hist (np.array): Histogram
        bins (np.array): Bin centers
        a_guess (float): Guess of a, as defined below
        x0_guess (float): Guess of x0, as defined in below
        sigma_guess (float): Guess of sigma, as defined below

    Return:
        a (float): a, as defined below
        x0 (float): x0, as defined below
        sigma (float): sigma, as defined below
        perr (np.array): Array with uncertainites on fit parameters
    """
    
    def Gaussian(x, a, x0, sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt, pcov = sci_opt.curve_fit(Gaussian, bins, hist, p0=[a_guess, x0_guess, sigma_guess])
    a, x0, sigma = popt[0], popt[1], abs(popt[2])
    perr = np.sqrt(np.diag(pcov))
    return a, x0, sigma, perr

# Get an approximation of the fit parameters, to aid the 'curve_fit' algorithm
def get_fit_parameters_guesses(hist, bins):
    """
    Function to estimate the parameters of a Gaussian.
    
    Args:
        hist (np.array): Histogram
        bins (np.array): Bin centers

    Return:
        a_guess (float): Guess of a, as defined in 'fit_data' above
        x0_guess (float): Guess of x0, as defined in 'fit_data' above
        sigma_guess (float): Guess of sigma, as defined in 'fit_data' above
    """
    
    def find_nearest(array, value):
        idx = (np.abs(array - value)).argmin()
        return idx
    
    # Extract relavant parameters
    maximum = max(hist)
    maximum_idx = find_nearest(hist, maximum)
    half_maximum = maximum/2
    half_maximum_idx_1 = find_nearest(hist[:maximum_idx], half_maximum)
    half_maximum_idx_2 = find_nearest(hist[maximum_idx:], half_maximum) + maximum_idx
    FWHM = bins[half_maximum_idx_2] - bins[half_maximum_idx_1]
    # Calculate guesses
    a_guess = maximum
    x0_guess = bins[maximum_idx]
    sigma_guess = FWHM/(2*np.sqrt(2*np.log(2)))
    return a_guess, x0_guess, sigma_guess


### Plotting

In [ ]:
def tof_histogram_plot(df, number_bins, label=None):
    """
    Function to plot a time-of-flight histogram.
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        number_bins (int): Number of bins in the histogram
        label (str): Label on data

    Yields:
        A histogram of time-of-flight
    
    Returns:
        hist (np.array): tof histogram
        bins (np.array): bin edges
    
    """
    
    # Get tof
    tof_in_us = df['tof']
    # Plot
    hist, bins, __ = plt.hist(tof_in_us, range=[0, 100000], bins=number_bins,
                              histtype='step', zorder=10, label=label
                              )
    plt.grid(True, which='major', linestyle='--', zorder=0)
    plt.grid(True, which='minor', linestyle='--', zorder=0)
    plt.yscale('log')
    plt.title('Time-of-flight histogram')
    plt.xlabel('tof (μs)')
    plt.ylabel('counts')
    return hist, bins
    

def energy_histogram_plot(tof_in_us, sample_to_detection_in_m, number_bins, label=None):
    """
    Function to plot an energy histogram.
    
    Args:
        tof_in_us (np.array): Time-of-flight values in micro-seconds
        sample_to_detection_in_m (np.array): Sample-to-detection distance in meters
        number_bins (int): Number of bins in histogra
        label (str): Label on data

    Yields:
        A histogram of energy
        
    Returns:
        hist (np.array): energy histogram
        bins (np.array): bin edges
    
    """
    
    # Get energies
    energies = get_energy(tof_in_us, sample_to_detection_in_m)
    # Plot
    hist, bins, __ = plt.hist(energies, range=[0, 30], bins=number_bins,
                              histtype='step', zorder=10
                              )
    plt.grid(True, which='major', linestyle='--', zorder=0)
    plt.grid(True, which='minor', linestyle='--', zorder=0)
    plt.yscale('log')
    plt.title('Energy histogram')
    plt.xlabel('E (meV)')
    plt.ylabel('counts')
    return hist, bins


def energy_transfer_plot(Ei_in_meV, tof_in_us, sample_to_detection_in_m, number_bins, label=None):
    """
    Function to plot an energy transfer histogram.
    
    Args:
        Ei_in_meV (float): Initial energy in meV
        tof_in_us (np.array): Time-of-flight in micro seconds
        sample_to_detection_in_m (np.array): Sample-to-detection distance in meters
        number_bins (int): Number of bins in histogra
        label (str): Label on data

    Yields:
        A histogram of energy transfer plot
        
    Returns:
        hist (np.array): energy transfer histogram
        bins (np.array): bin edges
    
    """
    
    # Get energy transfer data
    delta_E = get_energy_transfer(Ei_in_meV, tof_in_us, sample_to_detection_in_m)
    # Plot data
    delta_E_limit = Ei_in_meV * 0.2
    hist, bins, __ = plt.hist(delta_E, range=[-delta_E_limit, delta_E_limit],
                              bins=number_bins, histtype='step', zorder=10, label=label
                              )
    plt.grid(True, which='major', linestyle='--', zorder=0)
    plt.grid(True, which='minor', linestyle='--', zorder=0)
    plt.yscale('log')
    plt.title('Energy transfer histogram, E$_i$ = %.2f meV' % Ei_in_meV)
    plt.xlabel('E$_i$ - E$_f$ (meV)')
    plt.ylabel('counts')
    return hist, bins
    
def get_all_energy_transfer_plot(tof_in_us, sample_to_detection_in_m, number_bins, prominence=1e4):
    """
    Function to plot all energy transfer histogram.
    
    Args:
        tof_in_us (np.array): Time-of-flight in micro seconds
        sample_to_detection_in_m (np.array): Sample-to-detection distance in meters
        number_bins (int): Number of bins in histogra
        prominence (float): Peak prominence for the peak search algorithm to find

    Yields:
        All energy transfer histograms
    
    """
    
    
    
    # Produce an energy histogram over all energies
    hist, bin_edges = energy_histogram_plot(tof_in_us, sample_to_detection_in_m, 1000)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    # Get peaks
    peak_idxs = sci_sig.find_peaks(hist, prominence=prominence)[0]
    Eis_in_meV = bin_centers[peak_idxs]
    # Plot all energy transfer plots
    for Ei_in_meV in Eis_in_meV:
        fig = plt.figure()
        hist, bin_edges = energy_transfer_plot(Ei_in_meV, tof_in_us, sample_to_detection_in_m, number_bins)
        plt.show()
        


# 2. Data

## 2.1 Paths<a class="anchor" id="EXTRACT"></a>

## Multi-Grid detector

In [ ]:
MG_PATHS = {'mg_run_1': MG_RAW_FOLDER + 'mvmelst_212_200401_051232_SEQ1CaveShielded_SEQ2UnshieldedHorizontal_GasInAtTheStartOfTheRun_Continued_NoRHWGnd.zip',
            'mg_run_2': MG_RAW_FOLDER + 'ISIS_Source_FirstRun_mvmelst_241_201104_112406.zip',
            'mg_run_3': MG_RAW_FOLDER + 'ISIS_Source_2RUN_mvmelst_242_201104_115413.zip',
            'mg_run_4': MG_RAW_FOLDER + 'ISIS_Source_3RUN_mvmelst_243_201105_043444.zip',
            'mg_run_5': MG_RAW_FOLDER + 'BG_ISIS_mvmelst_244_201105_064156.zip'}

## Helium-3 tube

In [ ]:
HE3_PATHS = {'he3_run_1': HE3_FOLDER + 'LET00068153.nxs',
             'he3_run_2': HE3_FOLDER + 'LET00068806.nxs'}

## 2.2 Extract<a class="anchor" id="EXTRACT"></a>

## Multi-Grid detector

In [ ]:
# Note that the 'extract_and_save'-function only needs to performed once.
# After that, the data has been clustered and saved. The data can then
# be accessed using the 'load_clusters_and_events'-function.

#mg_read.extract_and_save('mg_run_1', MG_PATHS['mg_run_1'], MG_PROCESSED_FOLDER)
#mg_read.extract_and_save('mg_run_2', MG_PATHS['mg_run_2'], MG_PROCESSED_FOLDER)
#mg_read.extract_and_save('mg_run_3', MG_PATHS['mg_run_3'], MG_PROCESSED_FOLDER)
#mg_read.extract_and_save('mg_run_4', MG_PATHS['mg_run_4'], MG_PROCESSED_FOLDER)
#mg_read.extract_and_save('mg_run_5', MG_PATHS['mg_run_5'], MG_PROCESSED_FOLDER)

## Helium-3 tube

In [ ]:
# The data is in the format of NeXuS files, provided by the LET team.

## 2.3 Load<a class="anchor" id="LOAD"></a>

## Multi-Grid detector

In [ ]:
#mg_clu_run_1, mg_ev_run_1 = mg_read.load_clusters_and_events('mg_run_1', MG_PROCESSED_FOLDER)
mg_clu_run_2, mg_ev_run_2 = mg_read.load_clusters_and_events('mg_run_2', MG_PROCESSED_FOLDER)
mg_clu_run_3, mg_ev_run_3 = mg_read.load_clusters_and_events('mg_run_3', MG_PROCESSED_FOLDER)
mg_clu_run_4, mg_ev_run_4 = mg_read.load_clusters_and_events('mg_run_4', MG_PROCESSED_FOLDER)
mg_clu_run_5, mg_ev_run_5 = mg_read.load_clusters_and_events('mg_run_5', MG_PROCESSED_FOLDER)

## Helium-3 tubes

In [ ]:
he3_mapping_1 = he3_read.get_pixel_to_xyz_mapping(HE3_PATHS['he3_run_1'])
df_he3_run_1 = he3_read.import_data(HE3_PATHS['he3_run_1'])

# 3. Filters<a class="anchor" id="FILTER"></a>

Declare filters which can be used on the data. Below are a few template filters.

## Multi-Grid detector

In [ ]:
"""
Filters are declared in the following format: {'PARAMETER': [MIN_VALUE, MAX_VALUE, IS_ACTIVATE]}
"""

# Declare filters for SEQUOIA module
mg_filter_basic = {'wm': [1, 1, True],                   # Wire multiplicity
                   'gm': [1, 5, True],                   # Grid multiplicity
                   'wadc': [1000, np.inf, True],          # Wire charge
                   'gadc': [1000, np.inf, True],          # Grid charge
                   'tof': [0, np.inf, True],             # Time-of-flight (TDC channels)
                   'time': [0, np.inf, True],            # Time (TDC channels)
                   'bus': [0, 8, True],                  # Bus
                   'flag': [0, 1, False],                # =1 if different buses within same coincidence
                   'layer': [0, 19, False],              # Layer, front=0 to back=19
                   'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                   'gch': [80, 119, True]}               # Grid channel, bottom=80 to top=119     

mg_filter_none  = {'wm': [1, 1, False],                   # Wire multiplicity
                   'gm': [1, 5, False],                   # Grid multiplicity
                   'wadc': [600, np.inf, False],          # Wire charge
                   'gadc': [600, np.inf, False],          # Grid charge
                   'tof': [0, np.inf, False],             # Time-of-flight (TDC channels)
                   'time': [0, np.inf, False],            # Time (TDC channels)
                   'bus': [0, 8, False],                  # Bus
                   'flag': [0, 1, False],                # =1 if different buses within same coincidence
                   'layer': [0, 19, False],              # Layer, front=0 to back=19
                   'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                   'gch': [80, 119, False]}               # Grid channel, bottom=80 to top=119       

## Helium-3 tubes

In [ ]:
# Declare region of interest
pixels = []
start_pixel = 16384
height = 60
diff_tubes = 256
number_tubes = 32
for i in np.arange(0, 16, 1):
    start = start_pixel+diff_tubes*i
    stop = start_pixel+diff_tubes*i + height
    for j in np.arange(start, stop):
        pixels.append(j)
region_of_interest = np.array(pixels)

# 4. Plotting<a class="anchor" id="FILTER"></a>

## 4.1 Multi-Grid detector

### run_1: (test data from preparations at Utgård)

In [ ]:
area = 0.0225*0.0225*4*40
mg_plot_basic('run_1', mg_clu_run_1, mg_ev_run_1, mg_filter_basic, 0, 2, 1, area)

### run_2: (ISIS_Source_FirstRun_mvmelst_241_201104_112406.zip)

In [ ]:
area = 0.0225*0.0225*4*40
mg_plot_basic('run_2', mg_clu_run_2, mg_ev_run_2, mg_filter_basic, 0, 2, 1, area)

### run_3: (ISIS_Source_2RUN_mvmelst_242_201104_115413.zip)

In [ ]:
area = 0.0225*0.0225*4*40
mg_plot_basic('run_3', mg_clu_run_3, mg_ev_run_3, mg_filter_basic, 0, 2, 1, area)

### run_4: (ISIS_Source_3RUN_mvmelst_243_201105_043444.zip)

In [ ]:
area = 0.0225*0.0225*4*40
mg_plot_basic('run_4', mg_clu_run_4, mg_ev_run_4, mg_filter_basic, 0, 2, 1, area)

### run_5: (BG_ISIS_mvmelst_244_201105_064156.zip)

In [ ]:
area = 0.0225*0.0225*4*40
mg_plot_basic('run_5', mg_clu_run_5, mg_ev_run_5, mg_filter_basic, 0, 2, 1, area)

## 4.2 Helium-3 tubes

### Selection of region of interest

In [ ]:
he3_plot_coordinates(he3_mapping_1, region=region_of_interest)

### run_1: (test data sent from Ross)

In [ ]:
# Declare parameters
number_bins = 1000

# Define region of interest as a series of pixel id's
region_of_interest = np.arange(0, 98304, 1)

# Make geometrical cut
df_he3_run_1_cut = df_he3_run_1[df_he3_run_1['pixel_id'].isin(region_of_interest)]

# Plot
fig = plt.figure()
tof_histogram_plot(df_he3_run_1_cut, number_bins)
plt.show()
fig = plt.figure()
energy_histogram_plot(df_he3_run_1_cut['tof'],
                      he3_mapping_1['r'][df_he3_run_1_cut['pixel_id']],
                      number_bins)
plt.show()

# Get all energy transfer plots
get_all_energy_transfer_plot(df_he3_run_1_cut['tof'],
                             he3_mapping_1['r'][df_he3_run_1_cut['pixel_id']],
                             number_bins, prominence=1e4)